In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

NVDA_data = pd.read_csv('NVDA_data.csv')

train_data = NVDA_data[NVDA_data['Year'] != 2022].copy()
test_data = NVDA_data[NVDA_data['Year'] == 2022].copy()

# Train a logistic regression model on the training data
X_train = train_data[['Open', 'Adj Close', 'mean_return', 'volatility']]
y_train = train_data['color_label']
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on both training and testing sets
train_predictions = model.predict(X_train)
test_predictions = model.predict(test_data[['Open', 'Adj Close', 'mean_return', 'volatility']])

# Evaluate accuracy and confusion matrix
train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(test_data['color_label'], test_predictions)

train_conf_matrix = confusion_matrix(y_train, train_predictions, labels=['red', 'green'])
test_conf_matrix = confusion_matrix(test_data['color_label'], test_predictions, labels=['red', 'green'])

# Add predicted_label columns to the test_data DataFrame
test_data['predicted_label'] = test_predictions

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)
print("\nConfusion Matrix for Training Set:\n", train_conf_matrix)
print("\nConfusion Matrix for Testing Set:\n", test_conf_matrix)
print("\nTest DataFrame with Predicted Labels:\n", test_data.head())

Training Accuracy: 0.8530805687203792
Testing Accuracy: 0.8461538461538461

Confusion Matrix for Training Set:
 [[108  14]
 [ 17  72]]

Confusion Matrix for Testing Set:
 [[32  6]
 [ 2 12]]

Test DataFrame with Predicted Labels:
      Year  Week_Number        Open   Adj Close  mean_return  volatility  \
211  2022            1  298.149994  272.118683      -1.4652    3.573360   
212  2022            2  265.809998  269.072632      -0.1934    2.768013   
213  2022            3  262.600006  233.438583      -3.4885    0.320452   
214  2022            4  223.300003  228.105469      -0.4078    3.646720   
215  2022            5  231.820007  242.876404       1.3404    4.416611   

    color_label predicted_label  
211         red             red  
212         red             red  
213         red             red  
214         red             red  
215       green           green  


In [4]:
def trading_strategy_predicted_logistic_reg(df, initial_balance=100):
    df = df.reset_index(drop=True)  # Reset the index to numeric values
    balance = initial_balance
    position = None
    balance_history = []

    for week in range(len(df)):  # Adjust the loop condition
        open_price = df.loc[week, 'Open']
        close_price = df.loc[week, 'Adj Close']

        if df.loc[week, 'predicted_label'] == 'green':
            if position is None:
                shares_to_buy = balance / open_price
                print(f"Week {week}: Buying {shares_to_buy:.2f} shares at ${open_price:.2f}")
                position = shares_to_buy
                balance = 0
                print(f"   Shares: {position:.2f}, Portfolio Value: ${balance:.2f}")

        elif df.loc[week, 'predicted_label'] == 'red':
            if position is not None:
                balance_from_selling = position * df.loc[week - 1, 'Adj Close']
                print(f"Week {week}: Selling {position:.2f} shares at ${df.loc[week - 1, 'Adj Close']:.2f}, "
                      f"Portfolio Value: ${balance_from_selling:.2f}")
                balance += balance_from_selling
                position = None
                print(f"   Portfolio Value: ${balance:.2f}")

        if position is None:
            portfolio_value = balance
        else:
            portfolio_value = position * df.loc[week - 1, 'Adj Close']

        balance_history.append(portfolio_value)
        print(f"Week {week}: Portfolio Value: ${portfolio_value:.2f}")

    return balance_history

resulting_balance_history_lr = trading_strategy_predicted_logistic_reg(test_data)
print("Balance History based on Modified Predicted Labels for 2022:", resulting_balance_history_lr[-1])


Week 0: Portfolio Value: $100.00
Week 1: Portfolio Value: $100.00
Week 2: Portfolio Value: $100.00
Week 3: Portfolio Value: $100.00
Week 4: Buying 0.43 shares at $231.82
   Shares: 0.43, Portfolio Value: $0.00
Week 4: Portfolio Value: $98.40
Week 5: Selling 0.43 shares at $242.88, Portfolio Value: $104.77
   Portfolio Value: $104.77
Week 5: Portfolio Value: $104.77
Week 6: Portfolio Value: $104.77
Week 7: Buying 0.45 shares at $230.35
   Shares: 0.45, Portfolio Value: $0.00
Week 7: Portfolio Value: $107.39
Week 8: Selling 0.45 shares at $241.26, Portfolio Value: $109.73
   Portfolio Value: $109.73
Week 8: Portfolio Value: $109.73
Week 9: Portfolio Value: $109.73
Week 10: Buying 0.50 shares at $218.69
   Shares: 0.50, Portfolio Value: $0.00
Week 10: Portfolio Value: $110.77
Week 11: Portfolio Value: $132.58
Week 12: Selling 0.50 shares at $276.61, Portfolio Value: $138.79
   Portfolio Value: $138.79
Week 12: Portfolio Value: $138.79
Week 13: Portfolio Value: $138.79
Week 14: Portfolio V